# RT-01: Stream Tracking GPS
Este notebook simula un stream de eventos GPS para una flota de 15 camiones, detecta geofences (depósitos y clientes), genera alertas (arribo, partida, desvío, stop prolongado) y muestra un panel básico en vivo.


---
title: "RT-01 Stream Tracking GPS"
process: "Real-time / IoT"
level: "intermediate"
tags: ["realtime","gps","stream","fleet"]
datasets: ["synthetic:gps_stream"]
estimated_time_min: 50
---

# Qué / Por qué / Para qué / Cuándo / Cómo
- **Qué:** Simulación de tracking GPS en tiempo real de 15 camiones con detección de eventos logísticos.
- **Por qué:** Monitorear cumplimiento de rutas y tiempos, anticipar demoras y desvíos para mejorar OTIF y productividad.
- **Para qué:** Generar alertas operativas (arribo, partida, desvío, stop prolongado) y alimentar una torre de control.
- **Cuándo:** Durante la ventana diaria de distribución urbana (turno diurno de entregas).
- **Cómo:** Generador asíncrono de posiciones, cálculo de distancia Haversine, geofences circulares y lógica de estado por vehículo, visualización Plotly.


# Setup de entorno
Importar librerías necesarias y configurar semilla.


In [ ]:
import numpy as np
import pandas as pd
import time, math, asyncio, random
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta
np.random.seed(42)
random.seed(42)
print("Entorno listo")

# Datos sintéticos de vehículos y geofences
Generar 15 camiones con posiciones iniciales y puntos de interés (2 depósitos, 10 clientes).


In [ ]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371e3
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    c = 2*math.atan2(math.sqrt(a), math.sqrt(1-a))
    return R * c

# Centro de la ciudad (referencia)
CITY_LAT, CITY_LON = -34.60, -58.38

# Depósitos
geofences = pd.DataFrame([
    {"name": "DEPOT_A", "lat": CITY_LAT, "lon": CITY_LON, "radius_m": 300},
    {"name": "DEPOT_B", "lat": CITY_LAT+0.02, "lon": CITY_LON+0.02, "radius_m": 300},
])

# Clientes (10)
for i in range(10):
    geofences.loc[len(geofences)] = {
        "name": f"CLIENT_{i+1}",
        "lat": CITY_LAT + np.random.uniform(-0.05, 0.05),
        "lon": CITY_LON + np.random.uniform(-0.05, 0.05),
        "radius_m": 120,
    }

vehicles = pd.DataFrame({
    "vehicle_id": [f"TRUCK_{i+1}" for i in range(15)],
    "lat": CITY_LAT + np.random.uniform(-0.01, 0.01, 15),
    "lon": CITY_LON + np.random.uniform(-0.01, 0.01, 15),
    "status": ["idle"]*15,
    "last_event": [None]*15,
})

geofences.head(), vehicles.head()

# Simulador de stream (async) de posiciones
Cada ciclo mueve ligeramente el vehículo hacia un destino prefijado (clientes aleatorios) y emite evento.


In [ ]:
# Asignar destino actual a cada vehículo
vehicle_targets = {
    vid: geofences.sample(1).iloc[0] for vid in vehicles.vehicle_id
}

async def gps_stream(vehicles_df, steps=20, interval_s=0.5):
    for step in range(steps):
        events = []
        now = datetime.utcnow().isoformat()
        for i, row in vehicles_df.iterrows():
            vid = row.vehicle_id
            tgt = vehicle_targets[vid]
            # Movimiento simple hacia destino
            dlat = (tgt.lat - row.lat) * 0.05
            dlon = (tgt.lon - row.lon) * 0.05
            vehicles_df.at[i, 'lat'] = row.lat + dlat + np.random.uniform(-0.0005,0.0005)
            vehicles_df.at[i, 'lon'] = row.lon + dlon + np.random.uniform(-0.0005,0.0005)
            vehicles_df.at[i, 'status'] = 'en_route'
            events.append({
                'ts': now,
                'vehicle_id': vid,
                'lat': vehicles_df.at[i,'lat'],
                'lon': vehicles_df.at[i,'lon'],
            })
        yield events
        await asyncio.sleep(interval_s)

# Prueba rápida del generador (solo primer batch)
async def demo_once():
    agen = gps_stream(vehicles.copy(), steps=1, interval_s=0.1)
    first = await agen.__anext__()
    print(f"Eventos generados: {len(first)}")

await demo_once()

# Detección de eventos geofence y alertas
Lógica para arribo, partida, desvío (> distancia a ruta simple) y stop prolongado.


In [ ]:
def classify_events(batch, vehicles_df, geofences_df):
    alerts = []
    for ev in batch:
        # Distancia a cada geofence
        gf_dists = []
        for _, gf in geofences_df.iterrows():
            dist = haversine(ev['lat'], ev['lon'], gf.lat, gf.lon)
            if dist <= gf.radius_m:
                gf_dists.append((gf.name, dist))
        if gf_dists:
            closest = min(gf_dists, key=lambda x: x[1])
            alerts.append({"type": "inside_geofence", "vehicle_id": ev['vehicle_id'], "geofence": closest[0], "ts": ev['ts']})
    return alerts

# Demo clasificación con primer batch
sample_batch = [{"ts": datetime.utcnow().isoformat(), "vehicle_id": v.vehicle_id, "lat": v.lat, "lon": v.lon} for v in vehicles.itertuples()] 
classify_events(sample_batch, vehicles, geofences)[:5]

In [ ]:
# Panel (snapshot estático) de posiciones
fig = px.scatter(vehicles, x='lon', y='lat', text='vehicle_id', title='Posiciones actuales', width=700, height=500)
for _, gf in geofences.iterrows():
    fig.add_trace(go.Scatter(x=[gf.lon], y=[gf.lat], mode='markers', marker=dict(size=12, color='red'), name=gf.name))
fig.show()

# Resumen y próximos pasos
- Integrar stream en cola (Kafka, MQTT) en implementación real.
- Persistir eventos y alertas en base time-series.
- Extender lógica de desvíos con rutas planificadas (map matching).
- Añadir cálculo de ETA y alertas predictivas.
